Copyright (C) 2016 - 2019 Pinard Liu(liujianping-ok@163.com)

https://www.cnblogs.com/pinard

Permission given to modify the code as long as you keep this declaration at the top

scikit-learn 梯度提升树(GBDT)调参小结 https://www.cnblogs.com/pinard/p/6143927.html

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import model_selection, metrics
from sklearn.model_selection import GridSearchCV

import matplotlib.pylab as plt
%matplotlib inline

In [21]:
train = pd.read_csv('train_modified.csv')

train = pd.read_csv("aaa.tsv", sep = '\t')
print train
target='Disbursed' # Disbursed的值就是二元分类的输出
IDcol = 'ID'
train['Disbursed'].value_counts() 

       Disbursed  Existing_EMI           ID  Loan_Amount_Applied  \
0              0           0.0  ID000002C20               300000   
1              0           0.0  ID000004E40               200000   
2              0           0.0  ID000007H20               600000   
3              0           0.0  ID000008I30              1000000   
4              0       25000.0  ID000009J40               500000   
5              0       15000.0  ID000010K00               300000   
6              0           0.0  ID000011L10                    6   
7              0        2597.0  ID000012M20               200000   
8              0           0.0  ID000013N30                    0   
9              0           0.0  ID000014O40               300000   
10             0           0.0  ID000016Q10              1000000   
11             0           0.0  ID000018S30               140000   
12             0           0.0  ID000019T40               500000   
13             0        4600.0  ID000021V10     

0    19680
1      320
Name: Disbursed, dtype: int64

In [5]:
x_columns = [x for x in train.columns if x not in [target, IDcol]]
X = train[x_columns]
y = train['Disbursed']

In [20]:
gbm0 = GradientBoostingClassifier(random_state=10)
gbm0.fit(X,y)
y_pred = gbm0.predict(X)
print y_pred
print np.sum(y_pred)
y_predprob = gbm0.predict_proba(X)[:,1]
print "Accuracy : %.4g" % metrics.accuracy_score(y.values, y_pred)
print "AUC Score (Train): %f" % metrics.roc_auc_score(y, y_predprob)

[0 0 0 ... 0 0 0]
25
Accuracy : 0.9852
AUC Score (Train): 0.900531


In [22]:
param_test1 = {'n_estimators':range(20,81,10)}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=300,
                                  min_samples_leaf=20,max_depth=8,max_features='sqrt', subsample=0.8,random_state=10), 
                       param_grid = param_test1, scoring='roc_auc',iid=False,cv=5)
gsearch1.fit(X,y)
gsearch1.cv_results_['mean_test_score'], gsearch1.best_params_, gsearch1.best_score_, 

(array([0.81284735, 0.81437929, 0.81403709, 0.81592869, 0.81926607,
        0.81721608, 0.81485328]), {'n_estimators': 60}, 0.8192660696138212)

In [23]:
param_test2 = {'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200)}
gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60, min_samples_leaf=20, 
      max_features='sqrt', subsample=0.8, random_state=10), 
   param_grid = param_test2, scoring='roc_auc',iid=False, cv=5)
gsearch2.fit(X,y)
gsearch2.cv_results_['mean_test_score'], gsearch2.best_params_, gsearch2.best_score_, 

(array([0.81198869, 0.81267388, 0.81237654, 0.80924956, 0.81845981,
        0.81630145, 0.81314548, 0.81261631, 0.8180676 , 0.82137243,
        0.8168072 , 0.81383067, 0.81094266, 0.80967829, 0.81555712,
        0.81600927, 0.81164213, 0.81309427, 0.81693304, 0.81346505,
        0.81480008, 0.80911776, 0.81958802, 0.81382074]),
 {'max_depth': 7, 'min_samples_split': 300},
 0.8213724275914632)

In [10]:
param_test3 = {'min_samples_split':range(800,1900,200), 'min_samples_leaf':range(60,101,10)}
gsearch3 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60,max_depth=7,
                                     max_features='sqrt', subsample=0.8, random_state=10), 
                       param_grid = param_test3, scoring='roc_auc',iid=False, cv=5)
gsearch3.fit(X,y)
gsearch3.cv_results_['mean_test_score'], gsearch3.best_params_, gsearch3.best_score_, 

(array([0.81827919, 0.81731255, 0.8222033 , 0.8144698 , 0.81495371,
        0.81528439, 0.81590487, 0.81572702, 0.82021405, 0.81512044,
        0.81395095, 0.81586914, 0.82063564, 0.81489972, 0.82009218,
        0.81850308, 0.81855231, 0.81665754, 0.81960151, 0.81560198,
        0.81936055, 0.81361749, 0.81429473, 0.81299027, 0.81999889,
        0.82209294, 0.81820535, 0.81940422, 0.81544596, 0.8170426 ]),
 {'min_samples_leaf': 60, 'min_samples_split': 1200},
 0.8222032996697154)

In [11]:
gbm1 = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60,max_depth=7, min_samples_leaf =60, 
               min_samples_split =1200, max_features='sqrt', subsample=0.8, random_state=10)
gbm1.fit(X,y)
y_pred = gbm1.predict(X)
y_predprob = gbm1.predict_proba(X)[:,1]
print "Accuracy : %.4g" % metrics.accuracy_score(y.values, y_pred)
print "AUC Score (Train): %f" % metrics.roc_auc_score(y, y_predprob)

Accuracy : 0.984
AUC Score (Train): 0.908099


In [12]:
param_test4 = {'max_features':range(7,20,2)}
gsearch4 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60,max_depth=7, min_samples_leaf =60, 
               min_samples_split =1200, subsample=0.8, random_state=10), 
                       param_grid = param_test4, scoring='roc_auc',iid=False, cv=5)
gsearch4.fit(X,y)
gsearch4.cv_results_['mean_test_score'], gsearch4.best_params_, gsearch4.best_score_

(array([0.8222033 , 0.82241251, 0.82108184, 0.82064239, 0.82198258,
        0.81354802, 0.81876866]), {'max_features': 9}, 0.822412506351626)

In [13]:
param_test5 = {'subsample':[0.6,0.7,0.75,0.8,0.85,0.9]}
gsearch5 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60,max_depth=7, min_samples_leaf =60, 
               min_samples_split =1200, max_features=9, random_state=10), 
                       param_grid = param_test5, scoring='roc_auc',iid=False, cv=5)
gsearch5.fit(X,y)
gsearch5.cv_results_['mean_test_score'], gsearch5.best_params_, gsearch5.best_score_


(array([0.8182768 , 0.8234379 , 0.81673217, 0.82241251, 0.8228468 ,
        0.81738003]), {'subsample': 0.7}, 0.8234378969766262)

In [14]:
gbm2 = GradientBoostingClassifier(learning_rate=0.05, n_estimators=120,max_depth=7, min_samples_leaf =60, 
               min_samples_split =1200, max_features=9, subsample=0.7, random_state=10)
gbm2.fit(X,y)
y_pred = gbm2.predict(X)
y_predprob = gbm2.predict_proba(X)[:,1]
print "Accuracy : %.4g" % metrics.accuracy_score(y.values, y_pred)
print "AUC Score (Train): %f" % metrics.roc_auc_score(y, y_predprob)

Accuracy : 0.984
AUC Score (Train): 0.905324


In [15]:
gbm3 = GradientBoostingClassifier(learning_rate=0.01, n_estimators=600,max_depth=7, min_samples_leaf =60, 
               min_samples_split =1200, max_features=9, subsample=0.7, random_state=10)
gbm3.fit(X,y)
y_pred = gbm3.predict(X)
y_predprob = gbm3.predict_proba(X)[:,1]
print "Accuracy : %.4g" % metrics.accuracy_score(y.values, y_pred)
print "AUC Score (Train): %f" % metrics.roc_auc_score(y, y_predprob)

Accuracy : 0.984
AUC Score (Train): 0.908581


In [16]:
gbm4 = GradientBoostingClassifier(learning_rate=0.005, n_estimators=1200,max_depth=7, min_samples_leaf =60, 
               min_samples_split =1200, max_features=9, subsample=0.7, random_state=10)
gbm4.fit(X,y)
y_pred = gbm4.predict(X)
y_predprob = gbm4.predict_proba(X)[:,1]
print "Accuracy : %.4g" % metrics.accuracy_score(y.values, y_pred)
print "AUC Score (Train): %f" % metrics.roc_auc_score(y, y_predprob)

Accuracy : 0.984
AUC Score (Train): 0.908232


In [17]:
gbm11 = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60,max_depth=7, min_samples_leaf =60, 
               min_samples_split =1200, max_features=9, subsample=0.7, random_state=10)
gbm11.fit(X,y)
y_pred = gbm11.predict(X)
y_predprob = gbm11.predict_proba(X)[:,1]
print "Accuracy : %.4g" % metrics.accuracy_score(y.values, y_pred)
print "AUC Score (Train): %f" % metrics.roc_auc_score(y, y_predprob)

Accuracy : 0.984
AUC Score (Train): 0.899583
